In [ ]:
import os
import pandas as pd
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

# Solicitar o nome do arquivo ao usuário
nome_arquivo = input("Digite o nome do arquivo (sem extensão): ")

# Caminhos dos arquivos no diretório atual
excel_path = f"{nome_arquivo}.xlsx"
yaml_path = "competencias.yaml"

# Carregar o arquivo Excel
df = pd.read_excel(excel_path)

# Carregar o arquivo YAML de configuração
with open(yaml_path, "r") as file:
    config = yaml.safe_load(file)

# Descartar colunas conforme instruções do arquivo de configuração
discard_columns = config["csv_processing"]["discard_columns"]
df.drop(columns=discard_columns, inplace=True, errors='ignore')

# Extrair o nome da pessoa avaliada usando a variável definida no YAML
identificador_pergunta = config["avaliacao"]["pessoa_avaliada"]["identificador_pergunta"]
nome_avaliado_pattern = re.compile(r"Sua relação com (.+) é como:")
pessoa_avaliada = None

for col in df.columns:
    match = nome_avaliado_pattern.match(col)
    if match:
        pessoa_avaliada = match.group(1)
        break

# Verificar se o nome da pessoa avaliada foi encontrado
if pessoa_avaliada is None:
    raise ValueError("Não foi possível encontrar o nome da pessoa avaliada no arquivo Excel.")

# Criar pasta de saída com o nome da pessoa avaliada
output_dir = pessoa_avaliada
os.makedirs(output_dir, exist_ok=True)

# Configurações do heatmap
heatmap_config = config["heatmap_formatting"]

# Função para gerar heatmap
def gerar_heatmap(competencia_nome, perguntas_objetivas, df_filtered, pessoa_avaliada):
    # Preparar dados para o heatmap
    heatmap_data = df_filtered[perguntas_objetivas]
    heatmap_data.index = df_filtered["Tags"]
    
    # Ordenar para colocar autoavaliação no topo
    heatmap_data = heatmap_data.sort_values(by=["Tags"], key=lambda x: x == "autoavaliação", ascending=False)
    
    # Gerar heatmap
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(heatmap_data, cmap=heatmap_config["cmap"], cbar=heatmap_config["cbar"],
                     cbar_kws=heatmap_config["cbar_kws"], vmin=heatmap_config["vmin"], vmax=heatmap_config["vmax"],
                     yticklabels=heatmap_data.index, linewidths=heatmap_config["grid"]["linewidth"], linecolor=heatmap_config["grid"]["color"])
    plt.xticks(ticks=range(len(heatmap_config["index_labels"])), labels=heatmap_config["index_labels"])
    plt.xlabel(heatmap_config["xlabel"])
    plt.ylabel(heatmap_config["ylabel"])
    plt.title(f"Heatmap - {competencia_nome} ({pessoa_avaliada})")

    # Adicionar linha de separação para autoavaliação, se habilitado
    if heatmap_config.get("highlight_autoavaliacao", {}).get("enabled", False):
        autoavaliacao_idx = heatmap_data.index.get_loc("autoavaliação")
        ax.hlines([autoavaliacao_idx + 0.5], *ax.get_xlim(),
                  colors=heatmap_config["highlight_autoavaliacao"]["color"],
                  linestyles=heatmap_config["highlight_autoavaliacao"]["linestyle"],
                  linewidth=heatmap_config["highlight_autoavaliacao"]["linewidth"])

    # Salvar o heatmap
    plt.savefig(os.path.join(output_dir, f"Heatmap_{competencia_nome}.png"))
    plt.close()

# Função para gerar gráfico radar
def gerar_radar(df, competencias, pessoa_avaliada):
    categorias = [comp["nome"] for comp in competencias]
    num_categorias = len(categorias)
    
    # Calcular médias das notas recebidas de todos os avaliadores e autoavaliação
    medias_avaliadores = []
    medias_autoavaliacao = []
    
    for comp in competencias:
        perguntas = comp["perguntas"]["objetivas"]
        df_comp = df[["Tags"] + perguntas]
        
        # Filtrar respostas dos avaliadores e autoavaliação
        df_avaliadores = df_comp[df_comp["Tags"] != "autoavaliação"]
        df_autoavaliacao = df_comp[df_comp["Tags"] == "autoavaliação"]
        
        # Calcular médias
        media_avaliadores = df_avaliadores[perguntas].mean().mean()
        media_autoavaliacao = df_autoavaliacao[perguntas].mean().mean()
        
        # Adicionar médias às listas
        medias_avaliadores.append(media_avaliadores)
        medias_autoavaliacao.append(media_autoavaliacao)
        
        # Imprimir médias no console
        print(f"Competência: {comp['nome']}")
        print(f"Média dos Avaliadores: {media_avaliadores}")
        print(f"Média da Autoavaliação: {media_autoavaliacao}")
        print("-" * 50)
    
    # Criar gráfico radar
    angles = np.linspace(0, 2 * np.pi, num_categorias, endpoint=False).tolist()
    angles += angles[:1]
    
    medias_avaliadores += medias_avaliadores[:1]
    medias_autoavaliacao += medias_autoavaliacao[:1]
    
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.fill(angles, medias_avaliadores, color='blue', alpha=0.25)
    ax.fill(angles, medias_autoavaliacao, color='red', alpha=0.25)
    ax.plot(angles, medias_avaliadores, color='blue', linewidth=2, linestyle='solid', label='Avaliadores')
    ax.plot(angles, medias_autoavaliacao, color='red', linewidth=2, linestyle='solid', label='Autoavaliação')
    
    # Configurar a linha externa da circunferência
    ax.spines['polar'].set_visible(True)
    ax.spines['polar'].set_color('lightgray')
    
    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categorias)
    
    plt.title(f"Radar Chart - {pessoa_avaliada}", pad=20)
    plt.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
    
    # Salvar o radar plot
    plt.savefig(os.path.join(output_dir, "Radar_Chart.png"))
    plt.close()

# Iterar sobre todas as competências e gerar heatmaps
for competencia in config["avaliacao"]["competencias"]:
    competencia_nome = competencia["nome"]
    perguntas_objetivas = competencia["perguntas"]["objetivas"]
    avaliadores = competencia["avaliadores"] + ["autoavaliação"]
    
    # Identificar colunas de interesse
    colunas_interesse = perguntas_objetivas + ["Tags"]
    df_filtered = df[colunas_interesse]
    
    # Filtrar respostas dos avaliadores relevantes
    df_filtered = df_filtered[df_filtered["Tags"].isin(avaliadores)]
    
    # Converter respostas para valores numéricos se necessário
    if heatmap_config["apply_numeric_conversion"]:
        df_filtered[perguntas_objetivas] = df_filtered[perguntas_objetivas].apply(pd.to_numeric, errors='coerce')
    
    # Gerar heatmap para a competência atual
    gerar_heatmap(competencia_nome, perguntas_objetivas, df_filtered, pessoa_avaliada)

# Gerar gráfico radar para todas as competências
gerar_radar(df, config["avaliacao"]["competencias"], pessoa_avaliada)